# Financial Systems Design Homework 7
### Completed by: Rachel, Sarah, Peng Heng, Yu Kai

In [1]:
import ipyparallel as ipp
import multiprocessing as mp
import numpy as np
import math

rc = ipp.Client()
dview = rc[:]

## Q1
(a) Use the Standard Monte Carlo method to price the option. Recall that the price of the options is ~4.35% of X0 based on the previous homework assignment. Therefore, try experimenting with the lowest number of simulations and number of steps to get close to this value. Report your results.

In [2]:
def MC_CEV_Milstein_EKI_Call(S0,K,B,T,σ,r,β,it,steps):
    import math, numpy as np
    Δt = T/steps
    S = S0*np.ones(it)
    for t in range(steps):
        S += (r*S*Δt + σ*S**β*math.sqrt(Δt)*z[:,t] # updating the asset price to t+1 using Milstein scheme
              + 0.5*β*σ**2*S**(2*β-1)*Δt*(np.power(z[:,t],2)-1))
    KO = S>B
    V = math.exp(-r*T)*np.maximum(S-K,0)*KO
    return (np.average(V), np.var(V))

# define a function to reuse the code for calculating the combined sample average and standard error
def process_result(result, n_engs, N_per_eng):    
    px = [x[0] for x in result]   
    price = np.average(px)
    
    φ = sum(var[1] for var in result)*N_per_eng                   # sum of numerators of variance for each engine's sample
    for i in range(1, n_engs):                      
        φ += N_per_eng*i/(i+1)*(px[i] - np.average(px[0:i]))**2   # loop to add the incremental adjustment factors 

    stderr = math.sqrt((φ / (n_engs*N_per_eng-1)) / (n_engs*N_per_eng))  # calculate standard error using φ

    return price, stderr

In [3]:
n_engs = len(rc) # number of engines running
β = 0.75
S0 = 110.; K = 100.; B = 115.; T = 0.5; r = 0.02; σ = 0.25 # parameters
print(f'Expected Price {0.0435 * S0:.4}', )
print()
for N in [1000, 10000, 100000]:
    N_per_eng = int(N / n_engs) # number of iterations per engine
    for steps in [5, 10, 20, 30, 40]:
        print(N, 'simulations', 'using', steps, 'steps')

        prng = np.random.RandomState(42)
        Z = prng.standard_normal((N_per_eng*n_engs,steps))
        dview.scatter('z', Z)

        result = dview.apply_sync(MC_CEV_Milstein_EKI_Call,S0,K,B,T,σ,r,β,N_per_eng,steps)
        price, stderr = process_result(result, n_engs, N_per_eng)
        print('Milstein Scheme: Price = {:.4}'.format(price), 'standard error = {:.4}'.format(stderr))
        print()

Expected Price 4.785

1000 simulations using 5 steps
Milstein Scheme: Price = 4.759 standard error = 0.2612

1000 simulations using 10 steps
Milstein Scheme: Price = 4.66 standard error = 0.2579

1000 simulations using 20 steps
Milstein Scheme: Price = 4.542 standard error = 0.2592

1000 simulations using 30 steps
Milstein Scheme: Price = 4.568 standard error = 0.2594

1000 simulations using 40 steps
Milstein Scheme: Price = 4.583 standard error = 0.2576

10000 simulations using 5 steps
Milstein Scheme: Price = 4.838 standard error = 0.08311

10000 simulations using 10 steps
Milstein Scheme: Price = 4.847 standard error = 0.08376

10000 simulations using 20 steps
Milstein Scheme: Price = 4.8 standard error = 0.08318

10000 simulations using 30 steps
Milstein Scheme: Price = 4.739 standard error = 0.08291

10000 simulations using 40 steps
Milstein Scheme: Price = 4.816 standard error = 0.08279

100000 simulations using 5 steps
Milstein Scheme: Price = 4.744 standard error = 0.02613

100

### 100,000 simulations with 20 steps are sufficient to get within 0.03 difference from the expected price with a low standard error of ~0.026

## Q1
(b) Use the asset price, XT , as a control variable to price (a) again. Implement the algorithm where each parallel process returns the covariance matrix of the partial sample then calculate the covariance matrix of the combined sample using the piecewise formula given in the lecture slides to fully take advantage of parallel processing.

In [4]:
def control_var(S0,K,B,T,σ,r,β,it,steps):
    import math, numpy as np
    
    Δt = T/steps
    S = S0*np.ones(it)
    for t in range(steps):
        S += (r*S*Δt + σ*S**β*math.sqrt(Δt)*z[:,t] # updating the asset price to t+1 using Milstein scheme
              + 0.5*β*σ**2*S**(2*β-1)*Δt*(np.power(z[:,t],2)-1))

    avgSpot = np.average(S)

    KO = S>B
    V = math.exp(-r*T)*np.maximum(S-K,0)*KO
    
    price = np.average(V)
    cov = np.cov(S, V, ddof=0)
    # + adj*(avgSpot - S0*math.exp(r*T))
    
    return ((price, avgSpot), cov)

In [5]:
def process_control_var(result, n_engs, N_per_eng):
    X = [x[0][0] for x in result]
    Y = [x[0][1] for x in result]
    covs = [item[1] for item in result]
    φ = sum(cov[0, 1] for cov in covs) * N_per_eng
    for i in range(1, n_engs):
        φ += (i*N_per_eng)*N_per_eng / ((i+1)*N_per_eng) * (X[i] - np.average(X[0:i])) * (Y[i] - np.average(Y[0:i]))
    cov = φ / (n_engs*(N_per_eng-1))
    
    φ = sum(var[0, 0] for var in covs)*N_per_eng                   # sum of numerators of variance for each engine's sample
    for i in range(1, n_engs):                      
        φ += N_per_eng*i/(i+1)*(X[i] - np.average(X[0:i]))**2   # loop to add the incremental adjustment factors 

    var = (φ / (n_engs*N_per_eng-1))
    
    price = np.average(X)
    avgSpot = np.average(Y)

    return price, avgSpot, var, cov

In [6]:
n_engs = len(rc) # number of engines running
β = 0.75
S0 = 110.; K = 100.; B = 115.; T = 0.5; r = 0.02; σ = 0.25 # parameters
print(f'Expected Price {0.0435 * S0:.4}', )
print()
N = 100000
N_per_eng = int(N / n_engs) # number of iterations per engine
steps = 20
print(N, 'simulations', 'using', steps, 'steps')

prng = np.random.RandomState(42)
Z = prng.standard_normal((N_per_eng*n_engs,steps))
dview.scatter('z', Z)

result = dview.apply_sync(control_var,S0,K,B,T,σ,r,β,N_per_eng,steps)
price, avgSpot, var, cov = process_control_var(result, n_engs, N_per_eng)
adj = -cov/var
adj_price = price + adj*(avgSpot - S0*math.exp(r*T))
print('Price:', price)
print('Adjusted price:', adj_price)

Expected Price 4.785

100000 simulations using 20 steps
Price: 4.755175899956466
Adjusted price: 4.781660350722122


### From above, using asset price as a control variable reduces the amount of error from the estimate

## Q1
(c) Instead of the barrier being observed at maturity only, we will try and price the same option
but with the barrier observed continuously which is called American Knock-In (AKI) in the
industry. This means the option will knock in if the asset trades above the barrier at any point
during the lifetime of the option. The price is ~6.5% of X0. Therefore, try experimenting with
the lowest number of simulations and number of steps to get close to this value. How does
the value of N or number of steps required to get to the appropriate accuracy compare to the
EKI version of the option? (Hint: there are many possible ways to check the barrier condition
but one potential way which is more memory efficient is to keep track of the maximum of the
asset price in each simulation path).

In [7]:
def MC_CEV_Milstein_EKI_Call_Continuous(S0,K,B,T,σ,r,β,it,steps):
    import math, numpy as np
    Δt = T/steps
    S = S0*np.ones(it)
    barrier_cross_count = np.zeros_like(S)
    for t in range(steps):
        S += (r*S*Δt + σ*S**β*math.sqrt(Δt)*z[:,t] # updating the asset price to t+1 using Milstein scheme
              + 0.5*β*σ**2*S**(2*β-1)*Δt*(np.power(z[:,t],2)-1))
        barrier_cross_count += (S>B).astype(int)
    KO = barrier_cross_count > 0
    V = math.exp(-r*T)*np.maximum(S-K,0)*KO
    return (np.average(V), np.var(V))

In [8]:
n_engs = len(rc) # number of engines running
β = 0.75
S0 = 110.; K = 100.; B = 115.; T = 0.5; r = 0.02; σ = 0.25 # parameters
print(f'Expected Price {0.065 * S0:.4}', )
print()
for N in [10000, 100000]:
    N_per_eng = int(N / n_engs) # number of iterations per engine
    for steps in range(1000, 2001, 200):
        print(N, 'simulations', 'using', steps, 'steps')

        prng = np.random.RandomState(42)
        Z = prng.standard_normal((N_per_eng*n_engs,steps))
        dview.scatter('z', Z)

        result = dview.apply_sync(MC_CEV_Milstein_EKI_Call_Continuous,S0,K,B,T,σ,r,β,N_per_eng,steps)
        price, stderr = process_result(result, n_engs, N_per_eng)
        print('Milstein Scheme: Price = {:.4}'.format(price), 'standard error = {:.4}'.format(stderr))
        print()

Expected Price 7.15

10000 simulations using 1000 steps
Milstein Scheme: Price = 7.065 standard error = 0.08294

10000 simulations using 1200 steps
Milstein Scheme: Price = 6.969 standard error = 0.08299

10000 simulations using 1400 steps
Milstein Scheme: Price = 7.078 standard error = 0.08281

10000 simulations using 1600 steps
Milstein Scheme: Price = 7.138 standard error = 0.0835

10000 simulations using 1800 steps
Milstein Scheme: Price = 7.127 standard error = 0.08326

10000 simulations using 2000 steps
Milstein Scheme: Price = 7.1 standard error = 0.08307

100000 simulations using 1000 steps
Milstein Scheme: Price = 7.104 standard error = 0.02633

100000 simulations using 1200 steps
Milstein Scheme: Price = 7.108 standard error = 0.02632

100000 simulations using 1400 steps
Milstein Scheme: Price = 7.132 standard error = 0.02635

100000 simulations using 1600 steps
Milstein Scheme: Price = 7.131 standard error = 0.02637

100000 simulations using 1800 steps
Milstein Scheme: Price

### 100,000 simulations with 1800 steps are sufficient to get within 0.002 difference from the expected price with a low standard error of ~0.026